In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def generator_info(operator):
    """Provides the generator of a given operator.

    Args:
        operator (qml.ops): A PennyLane operator

    Returns:
        (qml.ops): The generator of the operator.
        (float): The coefficient of the generator.
    """
    gen = qml.generator(operator, format="observable")
    return gen.ops[0], gen.coeffs[0]

In [3]:
def derivative(op_order, params, diff_idx, wires, measured_wire):

    op_dict = {0: qml.RX, 1: qml.RY, 2: qml.RZ}
    dev = qml.device("default.qubit", wires=2)

    obs = qml.PauliZ(measured_wire)
    operator = op_dict[op_order[diff_idx]](params[diff_idx], wires[diff_idx])
    gen, coeff = generator_info(operator)

    @qml.qnode(dev)
    def circuit_bra1():
        for i in range(len(op_order)):
            if i == diff_idx:
                qml.apply(gen)
            op_dict[op_order[i]](params[i], wires[i])
        return qml.state()

    @qml.qnode(dev)
    def circuit_ket1():
        for i in range(len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(obs)
        return qml.state()

    @qml.qnode(dev)
    def circuit_bra2():
        for i in range(len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(obs)
        return qml.state()

    @qml.qnode(dev)
    def circuit_ket2():
        for i in range(len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])
            if i == diff_idx:
                qml.apply(gen)
        return qml.state()

    bra1 = circuit_bra1()
    ket1 = circuit_ket1()
    bra2 = circuit_bra2()
    ket2 = circuit_ket2()

    exp = coeff * ( np.vdot(1.j* bra1,ket1) + np.vdot(bra2,1.j*ket2 ) )
    exp = np.real(exp)
    return  exp

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    op_order, params, diff_idx, wires, measured_wire = json.loads(test_case_input)
    params = np.array(params, requires_grad=True)
    der = derivative(op_order, params, diff_idx, wires, measured_wire)
    return str(der)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your derivative isn't quite right!"

In [5]:
test_cases = [['[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]', '-0.2840528']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]'...
Correct!
